# MR Fingerprinting Dictionary Generation

In this example we demonstrate how to generate an MR Fingerprinting dictionary using a FISP type sequence

But first we have to install Julia (v1.10.6) on the current Colab Runtime. If I understood things correctly, the runtime can be used by multiple notebook files.

In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.10.6"
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [4]:
# Check julia installation
versioninfo()

Julia Version 1.10.6
Commit 67dffc4a8ae (2024-10-28 12:23 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
Threads: 1 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64


"/content"

In [ ]:
# Install external packages
using Pkg;

Pkg.add(["BenchmarkTools", "BlochSimulators", "ComputationalResources", "CUDA", "FFTW", "ImagePhantoms", "PythonPlot", "StaticArrays", "StructArrays"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ───────── v0.2.2
   Installed GPUArraysCore ─────────────── v0.1.6
   Installed Crayons ───────────────────── v4.1.1
   Installed MicroMamba ────────────────── v0.1.14
   Installed ColorTypes ────────────────── v0.11.5
   Installed Adapt ─────────────────────── v4.1.1
   Installed Scratch ───────────────────── v1.2.1
   Installed OffsetArrays ──────────────── v1.14.1
   Installed oneTBB_jll ────────────────── v2021.12.0+0
   Installed IntegerMathUtils ──────────── v0.1.2
   Installed TypeUtils ─────────────────── v1.4.0
   Installed Functors ──────────────────── v0.4.12
   Installed PythonCall ────────────────── v0.9.23
   Installed PrettyTables ──────────────── v2.4.0
   Installed demumble_jll ──────────────── v1.3.0+0
   Installed JSON3 ─────────────────────── v1.14.1
   Installed TableTraits ───────────────── v1.0.1
   Installed FFTW ──────────────────────── 

In [ ]:
# Load the necessary packages for this notebook
using BlochSimulators
using ComputationalResources
using StructArrays

## Simulation setup

In [ ]:
# Now construct a FISP sequence struct (see `src/sequences/fisp.jl`
# for which fields are necessary and which constructors exist)

nTR = 1000; # nr of TRs used in the simulation
RF_train = LinRange(1, 90, nTR) |> collect; # flip angle train
TR, TE, TI = 0.010, 0.005, 0.100; # repetition time, echo time, inversion delay
max_state = 64; # maximum number of configuration states to keep track of

sequence = FISP2D(RF_train, TR, TE, max_state, TI);

In [ ]:
# Next, set the desired input tissue properties for which the
# FISP sequence response will be simulated
T₁_range = logrange(0.1, 5.0, 50); # T₁ range
T₂_range = logrange(0.025, 0.5, 50); # T₂ range

# Generate valid combinations of T₁ and T₂ and store them in custom T₁T₂ struct
parameters = ([T₁T₂(T₁,T₂) for T₁ ∈ T₁_range, T₂ ∈ T₂_range if T₁ > T₂]);

println("Length parameters: $(length(parameters))")

# Now we can perform the simulations using different hardware resources

## Single-threaded CPU

In [ ]:
# Note that the first time a function is called in a Julia session,
# a precompilation procedure starts and the runtime for subsequent function
# calls are significantly faster
@time dictionary = simulate_magnetization(CPU1(), sequence, parameters);

In [ ]:
# The second time a function is called with arguments of similar types,
# the pre-compiled version is called immediatly.
@time dictionary = simulate_magnetization(CPU1(), sequence, parameters);

In [ ]:
# Note that the dictionary is a matrix with the magnetization response (at echo times)
# for all combinations of input tissue properties
@assert size(dictionary) == (nTR, length(parameters))

## Multi-threaded CPU

In [ ]:
# To use multiple threads, Julia must be started with the `--threads=auto`
# flag (or some integer instead of `auto`). Alternatively, set the
# environent variable `JULIA_NUM_THREADS` to the desired number of threads
# in your shell before starting Julia.

# Check the number of available threads
println("Current number of threads: $(Threads.nthreads())")

In [ ]:
# We can simulate in a multi-threaded fashion with the following syntax:
@time dictionary = simulate_magnetization(CPUThreads(), sequence, parameters);

In [ ]:
# In fact, BlochSimulators defaults to using CPUThreads() so we can also call
@time dictionary = simulate_magnetization(sequence, parameters);

## Distributed CPU

In [ ]:
# For distributed CPU mode, use the Distribute packages (ships with Julia)
# to add workers first

using Distributed
addprocs(4, exeflags="--project=.")

# Alternatively, if you can ssh into some other machine,
# you can add CPUs from that machine as follows:
# addprocs([("12.345.67.89", 4)], exeflags="--project=.")

# Or, if you want to run this code on cluster with a queuing system, use ClusterManagers package.
#
# After workers have been added, load BlochSimulators on all workers
# and then start a distributed dictionary generation with:
@everywhere using BlochSimulators

println("Current number of workers: $(nworkers())")
@time dictionary = simulate_magnetization(CPUProcesses(), sequence, parameters);

## GPU (CUDA device)

In [ ]:
# First, let's check if a CUDA device is available
println("Active CUDA device:");
BlochSimulators.CUDA.device();

# To perform simulations on GPU, we first convert the sequence and parameters
# to single precision and then send them to the gpu. To this end, BlochSimulators
# exports a `f32` function which recursively converts inputs to single precision.
# Similarly, a `gpu` function is exported which sends the input to the GPU.

cu_sequence = sequence |> f32 |> gpu;
cu_parameters = parameters |> f32 |> gpu;

# Remember, the first time a compilation procedure takes place which, especially
# on GPU, can take some time.
CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);

In [ ]:
# Call the pre-compiled version, it should be a lot faster
CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);

In [ ]:
# Now let's increase the number of tissue property combinations for which
# simulations are performed:
T₁ = rand(500_000)
T₂ = 0.1 * T₁
cu_parameters = (@parameters T₁ T₂) |> f32 |> gpu

CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);